IMPORTING LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install folium
import folium # map rendering library

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [2]:
#install PyDrive to pull in csv data
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 20.2MB/s 


In [0]:
#https://drive.google.com/file/d/1kzhizbGuRObxHRhoYjCSDqMCaxwHEaEz/view?usp=sharing

#download survey data from google drive
downloaded1 = drive.CreateFile({'id': '1kzhizbGuRObxHRhoYjCSDqMCaxwHEaEz'})
downloaded1.GetContentFile('Geospacial_Coordinates.csv')

In [5]:
# read csv file
Geospacial_Coordinates = pd.read_csv('Geospacial_Coordinates.csv', sep = ',') 
# examine the shape of original input data
print(Geospacial_Coordinates.shape)

(103, 3)


Create table - step 1 Use BeautifulSoup to scrape data from website:

In [0]:
#create an object with raw data from website
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [0]:
#create an object with the data from the website
soup = BeautifulSoup(website_url)

Create table - step 2 Based on the table in the website (search for table > extract to dict & create pd DataFrame)

In [0]:
#search for table
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table; #remove ';' to view output

Create table - step 3 Extract table data and create pd DataFrame

In [0]:
#extract row data to dict
row_data = []
for row in My_table.find_all("tr"):
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    row_data.append(cols)

row_data; #remove ';' to view output

# ***Part 1*** - Create initial table with  Postcodes ('Postcode', 'Borough', 'Neighborhood')

In [11]:
#create initial pd DataFrame
df_table = pd.DataFrame(row_data)
df_table = df_table.rename(columns={0:"Postcode",1:"Borough",2:"Neighborhood"})
df_table.head()

,Postcode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [12]:
#drop the first row (index = 0), and any row where 'Bourough' = 'Not assigned'
df_table2 = df_table.copy()
df_table2 = df_table.drop([0])
df_table2 = df_table2.drop(df_table2[df_table2['Borough']=='Not assigned'].index)
df_table2 = df_table2.reset_index(drop=True)
df_table2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Create table - step 4 - data transform - if 'Neighborhood' = 'Not Assigned', then use 'Borough'

In [13]:
#check a row where 'Neighborhood' = 'Not assigned'
df_table2.loc[6]

Postcode                 M7A
Borough         Queen's Park
Neighborhood    Not assigned
Name: 6, dtype: object

In [14]:
#create a new table and replace values if 'Neighborhood' = 'Not assigned' with 'Bourough'
df_table3 = df_table2.copy()

df_table3['Neighborhood'] = df_table3.apply(
    lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'],
    axis=1
)

#have a look at the transformed data
df_table3.loc[6]

Postcode                 M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 6, dtype: object

Create table - step 5 group the dataframe by Postcode & Borough and 'Join' values in 'Neighborhood'

In [15]:
df_table4 = df_table3.copy()

df_table4 = (df_table4.groupby(['Postcode','Borough'])['Neighborhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

df_table4 = pd.DataFrame(df_table4)
df_table4.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,West Hill,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
df_table4.shape

(103, 3)